# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,keflavik,64.0049,-22.5624,10.95,100,100,4.12,IS,1718917111
1,1,albany,42.6001,-73.9662,34.46,62,59,2.03,US,1718916915
2,2,wailua homesteads,22.0669,-159.3780,27.24,74,75,8.75,US,1718917113
3,3,souillac,-20.5167,57.5167,23.21,60,75,5.14,MU,1718917114
4,4,chibougamau,49.9168,-74.3659,20.79,37,20,6.17,CA,1718917115


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.scatter(
    x='Lng',
    y='Lat',
    c='Max Temp',
    cmap='viridis',
    colorbar=True,
    title='City Map',
    hover_cols=['City'],
    height=500,
    width=800
)

# Display the map
map_plot
%%capture --no-display

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [8]:
# Narrow down cities that fit criteria and drop any results with null values - I want to go somewhere cold
filtered_cities_df = city_data_df[(city_data_df['Max Temp'] >= -10) & (city_data_df['Max Temp'] <= 40)]

# Drop any rows with null values
filtered_cities_df = filtered_cities_df.dropna()

# Display sample data
filtered_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,keflavik,64.0049,-22.5624,10.95,100,100,4.12,IS,1718917111
1,1,albany,42.6001,-73.9662,34.46,62,59,2.03,US,1718916915
2,2,wailua homesteads,22.0669,-159.3780,27.24,74,75,8.75,US,1718917113
3,3,souillac,-20.5167,57.5167,23.21,60,75,5.14,MU,1718917114
4,4,chibougamau,49.9168,-74.3659,20.79,37,20,6.17,CA,1718917115


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,keflavik,IS,64.0049,-22.5624,100,
1,albany,US,42.6001,-73.9662,62,
2,wailua homesteads,US,22.0669,-159.3780,74,
3,souillac,MU,-20.5167,57.5167,60,
4,chibougamau,CA,49.9168,-74.3659,37,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation",  # Search only for accommodation (hotels)
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'
    params["bias"] = f'proximity:{lng},{lat}'

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
keflavik - nearest hotel: Núpan Deluxe
albany - nearest hotel: No hotel found
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
souillac - nearest hotel: Le Rochester
chibougamau - nearest hotel: Auberge chez Nancy
ugoofaaru - nearest hotel: No hotel found
blackmans bay - nearest hotel: Villa Howden
buala - nearest hotel: No hotel found
puerto baquerizo moreno - nearest hotel: Hostal D Estrellita
sur - nearest hotel: Sur Hotel
rumoi - nearest hotel: ホテル ニューホワイトハウス
adamstown - nearest hotel: No hotel found
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
petropavlovsk-kamchatsky - nearest hotel: No hotel found
kish - nearest hotel: هتل فارابی
remire-montjoly - nearest hotel: Motel du Lac
winnemucca - nearest hotel: The Overland Hotel
smithers - nearest hotel: Sunshine Inn Hotel
saipan - nearest hotel: Lavista
fredericton - nearest hotel: Days Inn
dhulia - nearest hotel: Plaza
margaret river - nearest hotel: No hotel found
carnarvon - neares

,City,Country,Lat,Lng,Humidity,Hotel Name
0,keflavik,IS,64.0049,-22.5624,100,Núpan Deluxe
1,albany,US,42.6001,-73.9662,62,No hotel found
2,wailua homesteads,US,22.0669,-159.3780,74,Hilton Garden Inn Kauai Wailua Bay
3,souillac,MU,-20.5167,57.5167,60,Le Rochester
4,chibougamau,CA,49.9168,-74.3659,37,Auberge chez Nancy
...,...,...,...,...,...,...
553,quezon,PH,7.7303,125.0989,97,Libre Inn
554,valdivia,CL,-39.8142,-73.2459,86,Esmeralda
555,marquette,US,46.5333,-87.6335,74,Wonderland Motel
556,sussex,US,41.1834,-74.6663,51,13


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [15]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.scatter(
    x='Lng',
    y='Lat',
    c='Humidity',
    cmap='viridis',
    colorbar=True,
    title='City Map with Hotel Information',
    hover_cols=['City', 'Country', 'Hotel Name'],
    height=500,
    width=800
)

# Display the map
map_plot
%%capture --no-display